In [ ]:
import matplotlib.pyplot as plt
import geopandas
import numpy as np

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import cartopy.crs as ccrs
import numpy as np

def scale_bar(ax, length=None, location=(0.5, 0.05), linewidth=3):
    """
    ax is the axes to draw the scalebar on.
    length is the length of the scalebar in km.
    location is center of the scalebar in axis coordinates.
    (ie. 0.5 is the middle of the plot)
    linewidth is the thickness of the scalebar.
    """
    #Get the limits of the axis in lat long
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    #Make tmc horizontally centred on the middle of the map,
    #vertically at scale bar location
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]
    tmc = ccrs.TransverseMercator(sbllx, sblly)
    #Get the extent of the plotted area in coordinates in metres
    x0, x1, y0, y1 = ax.get_extent(tmc)
    #Turn the specified scalebar location into coordinates in metres
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]

    #Calculate a scale bar length if none has been given
    #(Theres probably a more pythonic way of rounding the number but this works)
    if not length: 
        length = (x1 - x0) / 5000 #in km
        ndim = int(np.floor(np.log10(length))) #number of digits in number
        length = round(length, -ndim) #round to 1sf
        #Returns numbers starting with the list
        def scale_number(x):
            if str(x)[0] in ['1', '2', '5']: return int(x)        
            else: return scale_number(x - 10 ** ndim)
        length = scale_number(length) 

    #Generate the x coordinate for the ends of the scalebar
    bar_xs = [sbx - length * 500, sbx + length * 500]
    #Plot the scalebar
    ax.plot(bar_xs, [sby, sby], transform=tmc, color='k', linewidth=linewidth)
    #Plot the scalebar label
    ax.text(sbx, sby, str(length) + ' km', transform=tmc,
            horizontalalignment='center', verticalalignment='bottom')

In [ ]:
sigmine = geopandas.read_file("deps/RS/RS.shp")

In [ ]:
mapa_sigmine = {
'AREIA COMUM':'AREIA',
'AREIA FLUVIAL':'AREIA',
'AREIA QUARTZOSA':'AREIA',
'AREIA DE FUNDIÇÃO':'AREIA',
'AREIA INDUSTRIAL':'AREIA',
'ARGILA CAULÍNICA':'ARGILA',
'ARGILA REFRATÁRIA':'ARGILA',
'ARGILA VERMELHA':'ARGILA',
'ARENITO BETUMINOSO':'ARENITO',
'ARGILA P/CER. VERMELH':'ARGILA VERMELHA',
'BASALTO P/ BRITA':'BASALTO',
'BASALTO P/ REVESTIMENTO':'BASALTO',
'CALCÁRIO CALCÍTICO':'CALCÁRIO',
'CALCÁRIO CONCHÍFERO':'CALCÁRIO',
'CALCÁRIO DOLOMÍTICO':'CALCÁRIO',
'CALCÁRIO INDUSTRIAL':'CALCÁRIO',
'CARVÃO MINERAL':'CARVÃO',
'COBRE NATIVO':'COBRE',
'DIABÁSIO P/ BRITA':'DIABÁSIO',
'MINÉRIO DE CHUMBO':'CHUMBO',
'MINÉRIO DE COBRE':'COBRE',
'MINÉRIO DE OURO':'OURO',
'MINÉRIO DE TITÂNIO':'TITÂNIO',
'MINÉRIO DE ZINCO':'ZINCO',
'ÁGUA MINERAL ALC. BIC':'ÁGUA MINERAL',
'ÁGUA POTÁVEL DE MESA':'ÁGUA MINERAL'
}

sigmine['SUBS'] = sigmine['SUBS'].replace(mapa_sigmine)

In [ ]:
filtro_req1 = sigmine['FASE'] != 'REQUERIMENTO DE PESQUISA'
filtro_req2 = sigmine['FASE'] != 'REQUERIMENTO DE LICENCIAMENTO'
filtro_req3 = sigmine['FASE'] != 'REQUERIMENTO DE LAVRA GARIMPEIRA'
filtro_req4 = sigmine['FASE'] != 'REQUERIMENTO DE REGISTRO DE EXTRAÇÃO'

sigmine = sigmine[filtro_req1 & filtro_req2 & filtro_req3 & filtro_req4]

In [ ]:
filtro_n_cad = sigmine['SUBS'] != 'DADO NÃO CADASTRADO'
sigmine = sigmine[filtro_n_cad]

In [ ]:
sigmine.head()

In [ ]:
municipios = geopandas.read_file('deps/municipios/Municipios_IBGE.shp')

In [ ]:
municipios.head()

In [ ]:
lista_municipio = ['Caxias do Sul', 'Santa Maria', 'Porto Alegre', 'Pelotas']
def plot_municipios(municipios, lista_municipio, ax, c='black', delta=0.05):
    x = []
    y = []
    for index, row in municipios.iterrows():
        for m in lista_municipio:
            if row['NOME'] == m.upper():
                y.append(row['LATITUDESE'])
                x.append(row['LONGITUDES'])
    ax.scatter(x=x, y=y, marker='x', c=c)
    for i, txt in enumerate(lista_municipio[::-1]):
        ax.annotate(txt, (x[i]+delta, y[i]+delta), c=c)

In [ ]:
for mineral in sigmine["SUBS"].unique():
    municipios[mineral] = np.ones(len(municipios)) * float('nan')
    filtro = sigmine["SUBS"] == mineral
    filtrado = sigmine[filtro]
    geometrias = filtrado['geometry'].to_crs("EPSG:4326")
    centroides = geometrias.centroid
    for index, row in municipios.iterrows():
        poly = row['geometry']
        count = 0
        for p in centroides:
            if p.within(poly):
                count = count + 1
        #print(row['NOME'], mineral, count)
        municipios.at[index, mineral] = int(count)

In [ ]:
municipios.head()

In [ ]:
minerais = municipios.columns[12:]
#minerais = [municipios.columns[14]]

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

for mineral in minerais:
    if np.sum(municipios[mineral].values) > 30:
        fig, ax = plt.subplots(figsize=(12,12), subplot_kw={'projection': ccrs.PlateCarree()})
        municipios.plot(ax=ax, column=mineral, legend=False)

        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=1, color='gray', alpha=0.5, linestyle='--')
        #gl.xlabels_top = False

        divider = make_axes_locatable(ax)
        ax_cb = divider.new_horizontal(size="5%", pad=0.1, axes_class=plt.Axes)
        vmax = municipios[mineral].values.max()
        mappable = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=100))
        fig.add_axes(ax_cb)
        cbar = fig.colorbar(mappable, cax=ax_cb, ticks=[0, 50, 100])
        cbar.ax.set_yticklabels(['Baixa', 'Média', 'Alta'])

        #x, y, arrow_length = 0.04, 0.12, 0.1
        #ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
        #arrowprops=dict(facecolor='black', width=5, headwidth=15),
        #ha='center', va='center', fontsize=12,
        #xycoords=ax.transAxes)

        #scale_bar(ax, 100, location=(0.5,0.01))
        
        plot_municipios(municipios, lista_municipio, ax, c='gray', delta=0.05)

        ax.set_title('DENSIDADE DE PROCESSOS - {}'.format(mineral), loc='center', fontsize='14')
        nome = mineral.replace('/', '')
        plt.savefig('imagens/potencial/subs/'+nome+".png", bbox_inches='tight', facecolor='white', dpi = 300)
        #plt.show()
        
        ##############################################################################################################
        
        fig, ax = plt.subplots(figsize=(12,12), subplot_kw={'projection': ccrs.PlateCarree()})
        municipios.plot(ax=ax, color='gainsboro', edgecolor='silver')

        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=1, color='gray', alpha=0.5, linestyle='--')
        
        sigmine_minerais = sigmine[sigmine['SUBS'] == mineral]
        sigmine_minerais_disp = sigmine_minerais[sigmine['FASE']=='DISPONIBILIDADE']
        
        if len(sigmine_minerais_disp) > 0:
            
            x = []
            y = []
            for idx, row in sigmine_minerais_disp.iterrows():
                i = row['geometry'].centroid
                x.append(i.x)
                y.append(i.y)
                
            ax.scatter(x, y, color='green')
                
            #x, y, arrow_length = 0.04, 0.12, 0.1
            #ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            #arrowprops=dict(facecolor='black', width=5, headwidth=15),
            #ha='center', va='center', fontsize=12,
            #xycoords=ax.transAxes)

            #scale_bar(ax, 100, location=(0.5,0.01))

            plot_municipios(municipios, lista_municipio, ax, c='gray', delta=0.05)

            ax.set_title('ÁREAS DISPONÍVEIS - {}'.format(mineral), loc='center', fontsize='14')
            nome = mineral.replace('/', '')
            plt.savefig('imagens/disponiveis/disp_'+nome+".png", bbox_inches='tight', facecolor='white', dpi = 300)

In [ ]:
fig = plt.figure(figsize=(25,25))
nomes = []
alturas = []
for m in minerais:
    nomes.append(m)
    altura = len(sigmine['SUBS'][sigmine['SUBS']==m])
    alturas.append(altura)
    
alturas = np.array(alturas)
nomes = np.array(nomes)
inds = alturas.argsort()
nomes = nomes[inds]
alturas = alturas[inds]

x = [i for i in range(len(nomes))]
plt.xticks(x, nomes, rotation='vertical')
plt.yticks([i for i in range(50)])
plt.bar(x, alturas)
plt.ylim((0,50))
plt.grid()
plt.savefig("imagens/potencial/subs/processos.png", bbox_inches='tight', facecolor='white', dpi = 300)